In [1]:
# import os
# from dotenv import load_dotenv
# load_dotenv()
# import faiss
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import pandas as pd
import sqlite3
from langchain.prompts import ChatPromptTemplate

from langchain_ollama.llms import OllamaLLM

In [ ]:
SQL_DB_PATH = "../database/csv_database.db"
FAISS_INDEX_PATH = "../database/faiss_index"

In [3]:
# embedding function
def get_embeddings():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    return embeddings

In [4]:
# LLM model
llm = OllamaLLM(model="llama3.2")

In [5]:
# Fixed typo & improved formatting
PROMPT_TEMPLATE ="""
    Jawab pertanyaan berdasarkan konteks dari PDF berikut: 

    {context}

    ---

    **Pertanyaan:**
    {question}

    ---

    Jika pertanyaan terkait data riwayat penggunaan seperti:

    - "Jenis perangkat apa yang sering digunakan pengguna?"
    - "Kategori perintah apa yang sering digunakan pengguna?"
    - "Berapa rata-rata kepuasan pengguna sebelumnya?"
    - "Berapa persentase error yang dialami pengguna sebelumnya?"

    Jawablah dengan **SQL query** yang sesuai dengan contoh tabel berikut:

    **Nama Tabel:** `synthetic_data`

    **Contoh Data dalam Tabel (Format JSON, hanya contoh, tidak digunakan langsung dalam query):**
    ```json
    [
        {{"interaction_id":1,"user_id":447,"timestamp":"2024-03-12 17:56:48","device_type":"Smart Speaker","command_category":"Information","command_text":"Apa rekomendasi restoran di sekitar sini?","ai_response":"Akses ditolak","response_time_ms":310,"ai_confidence_score":67.97,"user_satisfaction":4,"status":"Error","error_code":"ERR403"}},
        {{"interaction_id":2,"user_id":469,"timestamp":"2024-12-01 06:40:13","device_type":"Smartphone","command_category":"Productivity","command_text":"Bagikan agenda meeting dengan anggota tim","ai_response":"Tugas telah disimpan","response_time_ms":315,"ai_confidence_score":70.04,"user_satisfaction":5,"status":"Success","error_code":null}}
    ]
    ```

    ---
    Hanya jawab query SQL, jika tidak ada data yang sesuai dalam konteks.
"""

PROMPT_DB = """
    Jawab pertanyaan berdasarkan hasil query SQL yang diberikan di bawah ini:
    {sql_query_result}

    ---
    Jawab pertanyaan berikut:
    {question}

    Jawab tanpa menyebutkan penggunaan query SQL, tetapi parafrase pertanyaan untuk memberi jawaban yang natural.
"""

In [6]:
def query_retrieval(question: str):
    # Load embeddings & vector store
    embeddings = get_embeddings()
    vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

    # Search vector store
    results = vector_store.similarity_search(question, k=5)
    if not results:
        return "Maaf, saya tidak menemukan informasi yang relevan dalam dokumen."

    context_text = '\n\n---\n\n'.join(doc.page_content for doc in results)

    # Prepare prompt with clear formatting
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)

    # Generate response using LLM
    response = llm.invoke(prompt)

    # Validate if response contains an SQL query
    if "SELECT" in response.upper():  
        try:
            # Connect to SQLite database
            conn = sqlite3.connect(SQL_DB_PATH)

            # Execute SQL query safely
            sql_query_result = pd.read_sql_query(response, conn)
            conn.close()

            # If DataFrame is empty, return error message
            if sql_query_result.empty:
                return "Query berhasil dijalankan, tetapi tidak ada hasil yang ditemukan."

            # Convert DataFrame to JSON format string
            query_result_str = sql_query_result.to_json(orient="records")

            # Prepare second-stage prompt (SQL result → natural language)
            prompt_db = ChatPromptTemplate.from_template(PROMPT_DB)
            final_prompt = prompt_db.format(sql_query_result=query_result_str, question=question)

            # Generate final response
            response = llm.invoke(final_prompt)

            return response
        
        except sqlite3.Error as e:
            return f"Terjadi kesalahan SQL: {e}"
    
    return response

In [7]:
# prompt
question = 'Berapa rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya?'
response = query_retrieval(question)
print(response)

Rata-rata kepuasan pengguna AI Assistant X-3000 sebelumnya adalah 3,463.


In [8]:
# prompt
question = 'Apa teknologi yang digunakan untuk pengenalan wajah AI Assistant X-3000?'
response = query_retrieval(question)
print(response)

Teknologi pengenalan wajah yang digunakan pada AI Assistant X-3000 adalah teknologi Facial Recognition. Teknologi ini memungkinkan AI Assistant untuk mengenali dan mengakses informasi dengan menggunakan wajah pengguna.

Dalam kasus AI Assistant X-3000, teknologi Facial Recognition dapat membantu AI Assistant untuk:

* Mengenali pengguna dan mengakses informasi yang relevan
* Mengontrol perangkat dengan menggunakan wajah pengguna
* Membuat pengalaman penggunaa yang lebih personal dan interaktif

Teknologi Facial Recognition juga dapat membantu meningkatkan keamanan dan privasi pengguna, karena hanya orang-orang yang memiliki wajah pengguna yang dapat mengakses informasi dan mengontrol perangkat.

Dalam keseluruhan, teknologi Facial Recognition pada AI Assistant X-3000 memungkinkan pengguna untuk berinteraksi dengan AI Assistant dalam cara yang lebih intuitif dan personal.
